In [1]:
import json
import pandas as pd
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from pandas import DataFrame
import sklearn.cluster as sk_cluster
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import pandas as pd
from pandas import DataFrame
import sklearn.feature_extraction.text as sk_text
import gensim 
from gensim.models import Word2Vec


In [2]:
business=pd.read_json('yelp_academic_dataset_business.json',lines=True)
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [3]:
data=business[(business.review_count >=10) & (business.city=='Toronto')]
business_ids=[]

for line in data.index:    
    if ('Shopping' in str(data.categories[line])  or'Bars' in str(data.categories[line]) or 'Beauty & Spas' in str(data.categories[line])):
        business_ids.append(data.business_id[line])
        
data=data[data['business_id'].isin(business_ids)]
data.head()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
161,cicPsia8Wj-DNRkmLbD_xg,The Keg Steakhouse + Bar,2201 Yonge Street,Toronto,ON,M4S 2B2,43.705843,-79.397841,3.5,91,1,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...","Steakhouses, Restaurants, Nightlife, Seafood, ...","{'Monday': '11:30-0:0', 'Tuesday': '11:30-0:0'..."
191,xVXyrTWbG8U3szze-aA7eg,St. Louis Bar & Grill,"557 Dixon Road, Unit 130",Toronto,ON,M9W 1H7,43.690144,-79.571658,2.0,34,1,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Nightlife, Sports Bars, Bars, Restaurants, Ame...","{'Monday': '11:30-0:0', 'Tuesday': '11:30-0:0'..."
194,e-tRKAC-q40SqQfAOwYa-A,Aum Ayurvedic & Thai Massage,538 College Street,Toronto,ON,M6G 1A6,43.656008,-79.411205,3.0,12,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Reflexology, Massage Therapy, Massage, Beauty ...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
247,TKKX9bllRBMzp0NYr441PQ,Mi'Hito Sushi Laboratory,285 College Street,Toronto,ON,M5T 1S2,43.657621,-79.400885,4.0,51,1,"{'Caters': 'False', 'HasTV': 'True', 'Restaura...","Food, Poke, Fast Food, Sushi Bars, Restaurants...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:..."
259,C9keC4mWuXdl2mYFHZXudQ,Uncle Otis Clothing,329 Spadina Ave,Toronto,ON,M5T 2E9,43.654631,-79.398246,5.0,12,1,"{'BusinessParking': '{'garage': False, 'street...","Men's Clothing, Fashion, Shopping, Sports Wear...","{'Monday': '11:0-19:0', 'Tuesday': '11:0-19:0'..."


In [4]:
textReview=pd.read_csv('outputforex3.csv',usecols=['business_id','textReview'])
textReview

,business_id,textReview
0,9Eghhu_LzEJgDKNgisf3rg,Was super excited to bring my boyfriend to thi...
1,cDoo0Pf1d_P79Rq3ZNk-Mw,"Sigh. This review was tough to give, as it cou..."
2,4m_hApwQ054v3ue_OxFmGw,"Excellent food, service and ambiance! The tapa..."
3,KWojrEKjUqHCRI5HO7pB4g,After my scathing review of the Perez Museum i...
4,NrJSy3dgcXErFtOKGREmnw,Second brunch visit. Loved the Hash and the Eg...
...,...,...
108937,UxWH8zRYIBgs6Q2oykvRdw,Always enjoy my brunches here. Love the oyster...
108938,mCNzKFTukMYHkZVRMXIIyw,This is probably the worst restaurant.. ever. ...
108939,YPwOvJRVAjlhChzsGnCdyg,"Small cozy place . Great prices, really good s..."
108940,GhvZDqLJApNYFrByjnb2cA,""":""Where did these boobies come from? They're ..."


In [5]:
reviews=textReview.groupby(['business_id'])



In [6]:

cl_data=data.drop(['name','address','city','state','postal_code','latitude','longitude','is_open','attributes','hours'],axis=1)

for line in cl_data.index:    
    if 'Shopping' in str(cl_data.categories[line]) :
      cl_data.loc[line, 'categories']='Shopping'

    elif 'Bars' in str(cl_data.categories[line]) :
      cl_data.loc[line, 'categories']='Bars'
      
    elif 'Beauty & Spas' in str(cl_data.categories[line]):
      cl_data.loc[line, 'categories']='Beauty & Spas'



In [7]:
cl_data['categories'].replace({"Bars":"1","Shopping":"2","Beauty & Spas":"3"},inplace=True)
cl_data.to_csv("cl_data.csv")

In [8]:
cl_data=pd.merge(cl_data,reviews.sum(),how='inner',on="business_id")
cl_data

,business_id,stars,review_count,categories,textReview
0,cicPsia8Wj-DNRkmLbD_xg,3.5,91,1,"Consistently good, as the Keg tends to be.\n\n..."
1,xVXyrTWbG8U3szze-aA7eg,2.0,34,1,I would give zero stars. I came here with a gr...
2,TKKX9bllRBMzp0NYr441PQ,4.0,51,1,Mi'Hito is a new poke\/sushi burrito chain tha...
3,C9keC4mWuXdl2mYFHZXudQ,5.0,12,2,If you're a boy and you want to wear some hot ...
4,LRRhbtaohbXGMdaJ1yZjbQ,4.0,38,1,This place came highly recommended by a collea...
...,...,...,...,...,...
1959,__8j8yhsmE98wNWHJNyAgw,3.0,91,1,I got food poisoning from here.\n\nThe sushi w...
1960,VIsPmeZBMLkgbgw4UqCjUQ,3.0,12,1,Having heard about AAA from others and undoubt...
1961,BagD2-u3trnlefs_OEPIig,4.5,18,1,"Milka, the owner, is an absolute delight-grab ..."
1962,Ik7WA5Lb6zUQ6zPxCxPCJQ,2.5,16,2,I've been a member for about four years and at...


In [9]:
reviews_data=[]
for i in cl_data["textReview"]:
    reviews_data.append(i)


vectorizer = sk_text.TfidfVectorizer(stop_words='english',
                             max_features = 1000,
                             min_df=4, max_df=0.8)
X= vectorizer.fit_transform(reviews_data)
y=cl_data["categories"]




In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)
display(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
print(X_train)

(1473, 1000)

(1473,)

(491, 1000)

(491,)

  (0, 36)	0.31707152213080675
  (0, 386)	0.04539200745735939
  (0, 246)	0.7018767688551377
  (0, 128)	0.018058671288356917
  (0, 969)	0.015186051319343261
  (0, 971)	0.016459986063473476
  (0, 631)	0.01563331917253262
  (0, 937)	0.0165669636175872
  (0, 845)	0.013813513351600632
  (0, 92)	0.03278784800384329
  (0, 332)	0.011212725815725631
  (0, 160)	0.014278422990841456
  (0, 637)	0.016926053137454225
  (0, 138)	0.014278422990841456
  (0, 464)	0.013428901394608252
  (0, 575)	0.030681982396227183
  (0, 777)	0.06495122813630608
  (0, 633)	0.0277192000863329
  (0, 310)	0.01538903334709787
  (0, 241)	0.02925982955464904
  (0, 509)	0.015788808855080397
  (0, 836)	0.0323220017631659
  (0, 634)	0.03725520739133302
  (0, 619)	0.015492713238541223
  (0, 152)	0.012677025757741676
  :	:
  (1472, 827)	0.03266813109652407
  (1472, 187)	0.028799809688689965
  (1472, 629)	0.038078184208888984
  (1472, 697)	0.03793404739483772
  (1472, 894)	0.10807269948469607
  (1472, 260)	0.09301413496548501
  (147

In [11]:
dtree = tree.DecisionTreeClassifier()
dtree = dtree.fit(X_train, y_train)

print("classifier accuracy:",dtree.score(X_test,y_test))

y_pred = dtree.predict(X_test)
y_prob = dtree.predict_proba(X_test)

classifier accuracy: 0.9368635437881874


In [12]:

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
print("classifier score:", knn.score(X_test,y_test))

y_pred = knn.predict(X_test)

print("\naccuracy:",metrics.accuracy_score(y_test,y_pred))
print("\nConfusion matrix")
print(metrics.confusion_matrix(y_test,y_pred))
print("\nPrecision Score per class")
print(metrics.precision_score(y_test,y_pred,average=None))
print("\nAverage Precision Score")
print(metrics.precision_score(y_test,y_pred,average='weighted'))
print("\nRecall Score per class")
print(metrics.recall_score(y_test,y_pred,average=None))
print("\nAverage Recall Score")
print(metrics.recall_score(y_test,y_pred,average='weighted'))
print("\nF1-score Score per class")
print(metrics.f1_score(y_test,y_pred,average=None))
print("\nAverage F1 Score")
print(metrics.f1_score(y_test,y_pred,average='weighted'))



classifier score: 0.9531568228105907

accuracy: 0.9531568228105907

Confusion matrix
[[318   1]
 [ 22 150]]

Precision Score per class
[0.93529412 0.99337748]

Average Precision Score
0.9556410400849892

Recall Score per class
[0.9968652  0.87209302]

Average Recall Score
0.9531568228105907

F1-score Score per class
[0.96509863 0.92879257]

Average F1 Score
0.9523804202063811


In [13]:
svm_clf = svm.SVC()
svm_clf.fit(X_train,y_train)
print("classifier score:",svm_clf.score(X_test,y_test))
y_pred = svm_clf.predict(X_test)
print("\naccuracy:",metrics.accuracy_score(y_test,y_pred))
print("\nConfusion matrix")
print(metrics.confusion_matrix(y_test,y_pred))
print("\nPrecision Score per class")
print(metrics.precision_score(y_test,y_pred,average=None))
print("\nAverage Precision Score")
print(metrics.precision_score(y_test,y_pred,average='weighted'))
print("\nRecall Score per class")
print(metrics.recall_score(y_test,y_pred,average=None))
print("\nAverage Recall Score")
print(metrics.recall_score(y_test,y_pred,average='weighted'))
print("\nF1-score Score per class")
print(metrics.f1_score(y_test,y_pred,average=None))
print("\nAverage F1 Score")
print(metrics.f1_score(y_test,y_pred,average='weighted'))

classifier score: 0.955193482688391

accuracy: 0.955193482688391

Confusion matrix
[[317   2]
 [ 20 152]]

Precision Score per class
[0.94065282 0.98701299]

Average Precision Score
0.9568930407828798

Recall Score per class
[0.99373041 0.88372093]

Average Recall Score
0.955193482688391

F1-score Score per class
[0.96646341 0.93251534]

Average F1 Score
0.9545712165073371


In [14]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
y_pred
y_test

18      1
781     1
342     2
1273    2
579     1
       ..
1474    2
302     2
532     2
1036    2
171     1
Name: categories, Length: 491, dtype: object

In [15]:
cnf_matrix=metrics.confusion_matrix(y_test,y_pred)
cnf_matrix

array([[317,   2],
       [ 18, 154]])

In [16]:
accuracyScore=metrics.accuracy_score(y_test,y_pred)
print('accuracy score:',accuracyScore)
print("accuracy in percentage:",int(accuracyScore*100),"%")
print("\nPrecision Score per class")
print(metrics.precision_score(y_test,y_pred,average=None))
print("\nRecall Score per class")
print(metrics.recall_score(y_test,y_pred,average=None))
print("\nF1-score Score per class")
print(metrics.f1_score(y_test,y_pred,average=None))

accuracy score: 0.9592668024439919
accuracy in percentage: 95 %

Precision Score per class
[0.94626866 0.98717949]

Recall Score per class
[0.99373041 0.89534884]

F1-score Score per class
[0.96941896 0.93902439]


In [17]:
import sklearn.model_selection as model_selection

scores = model_selection.cross_val_score(
                                          dtree,
                                          X,
                                          y,
                                          scoring='f1_weighted',
                                          cv=5)
print ("Dtree scores:",scores)
print (scores.mean())

Dtree scores: [0.93134746 0.93402336 0.91868909 0.92830804 0.93894598]
0.9302627831517171


In [18]:
scores2 = model_selection.cross_val_score(
                                          lr,
                                          X,
                                          y,
                                          scoring='f1_weighted',
                                          cv=5)
print (" Lr scores:",scores2)
print (scores2.mean())

 Lr scores: [0.98208989 0.96927487 0.97693656 0.95881377 0.96405896]
0.9702348103228011


In [19]:
scores3 = model_selection.cross_val_score(
                                          svm_clf,
                                          X,
                                          y,
                                          scoring='f1_weighted',
                                          cv=5)
print ("Svm scores:",scores3)
print (scores3.mean())

Svm scores: [0.97951388 0.97701263 0.96402849 0.96402849 0.9590595 ]
0.9687285973342628


In [20]:
scores4 = model_selection.cross_val_score(
                                          knn,
                                          X,
                                          y,
                                          scoring='f1_weighted',
                                          cv=5)
print ("Knn scores:",scores4)
print (scores4.mean())

Knn scores: [0.97443547 0.97190432 0.96396205 0.94861213 0.94630765]
0.9610443244819671
